# PRiAD - Projekt zespołowy "Klasyfikacja wniosków kredytowych"

---

In [2]:
# Import potrzebnych bibliotek

import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as pat
from skimage import io
import skimage as si
import skimage.filters as sif
import skimage.segmentation as sis
import skimage.measure as sime
import skimage.morphology as simo
import skimage.color as sic

from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

# Zmiana sposobu wyświetlania danych zmiennoprzecinkowych
pd.options.display.float_format = "{:.2f}".format 

In [21]:
# Przydatne funkcje

def divide(data, proportion = 0.3):
    opis_ucz, opis_test, dec_ucz, dec_test = train_test_split(
        data.iloc[:,0:-1], 
        data.iloc[:,-1].astype('category').cat.codes, 
        test_size = proportion
    )

    return {
        "opis_ucz":opis_ucz, 
        "opis_test":opis_test, 
        "dec_ucz":dec_ucz, 
        "dec_test":dec_test
    }


def getErrorMatrix(data, testResult):
    return pd.crosstab(data["dec_test"], testResult) 


def getTotalErrors(errorMatrix):
    totalErrors = 0
    # Errors above diagonal
    for row in range(0, errorMatrix.shape[0]):
        for column in range(row + 1, errorMatrix.shape[1]):
            totalErrors += errorMatrix.iloc[row, column]
    # Errors below diagonal
    for column in range(0, errorMatrix.shape[1]):
        for row in range(column + 1, errorMatrix.shape[0]):
            totalErrors += errorMatrix.iloc[row, column]

    return totalErrors


def verify(data, model, attributes):
    model.fit(
        data["opis_ucz"].iloc[:, attributes], 
        data["dec_ucz"]
    )
    
    testResult = model.predict(
        data["opis_test"].iloc[:, attributes]
    )

    testErrorMatrix = getErrorMatrix(data, testResult)
    print("Macierz pomyłek dla zbioru testowego:")
    print(testErrorMatrix)

    totalErrors = getTotalErrors(testErrorMatrix)
    print("Suma błędów zbioru testowego: %d" % totalErrors)

In [56]:
# Konwersja .data --> .csv do typu Data Frame

data = pd.read_csv("crx.data", header = None, sep = ",")
data = data.dropna()
data = data.rename(columns={0:'A1', 1:'A2', 2:'A3', 3:'A4', 4:'A5', 5:'A6', 6:'A7', 7:'A8', 8:'A9',
                           9:'A10', 10:'A11', 11:'A12', 12:'A13', 13:'A14', 14:'A15', 15:'A16'})
data['A1'] = data['A1'].astype('category').cat.codes
data['A4'] = data['A4'].astype('category').cat.codes
data['A5'] = data['A5'].astype('category').cat.codes
data['A6'] = data['A6'].astype('category').cat.codes
data['A7'] = data['A7'].astype('category').cat.codes
data['A9'] = data['A9'].astype('category').cat.codes
data['A10'] = data['A10'].astype('category').cat.codes
data['A11'] = data['A11'].astype('category').cat.codes
data['A13'] = data['A13'].astype('category').cat.codes
data['A15'] = data['A15'].astype('category').cat.codes
# cecha programowania obiektowego ctr c + ctr v
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,30.83,0.00,2,1,13,8,1.25,1,1,1,f,0,00202,0,+
1,1,58.67,4.46,2,1,11,4,3.04,1,1,6,f,0,00043,119,+
2,1,24.50,0.50,2,1,11,4,1.50,1,0,0,f,0,00280,140,+
3,2,27.83,1.54,2,1,13,8,3.75,1,1,5,t,0,00100,3,+
4,2,20.17,5.62,2,1,13,8,1.71,1,0,0,f,2,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,2,21.08,10.09,3,3,5,4,1.25,0,0,0,f,0,00260,0,-
686,1,22.67,0.75,2,1,2,8,2.00,0,1,2,t,0,00200,106,-
687,1,25.25,13.50,3,3,6,3,2.00,0,1,1,t,0,00200,1,-
688,2,17.92,0.20,2,1,1,8,0.04,0,0,0,f,0,00280,134,-


In [23]:
# Zakodowanie danych
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
        transformers = [('one_hot_encoder', OneHotEncoder(categories = 'auto'),[3, 4, 5, 11])],
        remainder = 'passthrough')

X = ct.fit_transform(X)

In [24]:
# Standaryzacja danych
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

ValueError: could not convert string to float: 'b'

In [ ]:
# Budowa klasyfikatora
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 0)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 170, random_state = 0)
classifier.fit(X_train, y_train)

# Predictions
y_pred = classifier.predict(X_test)

# Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

# K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

print("\nMean of the Accuracies after cross-validation: ", accuracies.mean())
print("\nStandard Deviation within the accuracies: ", accuracies.std())

print('\nAccuracy: ', accuracy)

In [ ]:
# Histogramy wszystkich kolumn


